In [5]:
import torch
import torchvision
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch 
import torch.nn as nn 
import torch.optim as optim 
import torchvision 
import torchvision.transforms as transforms 
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.dataset import random_split
from torchvision.datasets import ImageFolder

In [3]:
# 画像をテンソルに変換
data_transform = transforms.Compose([
    transforms.ToTensor()
])

In [10]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Grayscale(num_output_channels=3),  # 1チャネルから3チャネルに変換
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],  # ImageNetの平均
                            [0.229, 0.224, 0.225])  # ImageNetの標準偏差
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Grayscale(num_output_channels=3),  # 1チャネルから3チャネルに変換
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],  # ImageNetの平均
                            [0.229, 0.224, 0.225])  # ImageNetの標準偏差
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Grayscale(num_output_channels=3),  # 1チャネルから3チャネルに変換
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],  # ImageNetの平均
                            [0.229, 0.224, 0.225])  # ImageNetの標準偏差
    ]),
}


In [12]:
full_dataset = ImageFolder(root='/Users/serenakurashina/ess4-3/Cable1_Dataset')

# データセットを70%訓練、15%検証、15%テストに分割
train_size = int(0.7 * len(full_dataset))
val_size = int(0.15 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [train_size, val_size, test_size]
)

train_dataset.dataset.transform = transforms.Compose(data_transforms)
val_dataset.dataset.transform = transforms.Compose(data_transforms)
test_dataset.dataset.transform = transforms.Compose(data_transforms)

In [13]:
# ハードウェアの設定 
device = torch.device('mps')

In [15]:
# DataLoader の定義
BATCH_SIZE = 64
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [16]:
# ニューラルネットワークに入力する画像のサイズ、チャネル数をチェック
images, labels = next(iter(train_loader))
c, h, w = images[0].shape
print("ミニバッチサイズ: ", images.size())
print("画像のチャネル数: ", c)
print("画像の高さ: ", h)
print("画像の幅: ", w)

# グリッド乗に4枚の画像を表示
img = torchvision.utils.make_grid(images[:4])
img = transforms.functional.to_pil_image(img)
display(img)

# 上の画像に対応するラベルを表示
print(labels[:4])

TypeError: 'str' object is not callable